<a href="https://colab.research.google.com/github/Shivcoderrr/Shivcoderrr/blob/main/TEXT_CLASSFIER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Create a dummy DataFrame
data = {'text': ["This is a positive review.",
                 "I did not like this product.",
                 "The service was excellent!",
                 "Terrible experience overall.",
                 "It was just okay."],
        'label': [1, 0, 1, 0, 0]}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("dataset.csv", index=False)

In [3]:
# Load dataset
data = pd.read_csv("dataset.csv")
texts = data['text'].astype(str).tolist()
labels = data['label'].tolist()

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Tokenization
vocab_size = 10000
max_length = 100
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

# Model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.GlobalMaxPool1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Save model and tokenizer
model.save("text_classifier.h5")
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# Predict function
def predict_text(text):
    with open("tokenizer.pkl", "rb") as f:
        tokenizer = pickle.load(f)
    with open("label_encoder.pkl", "rb") as f:
        label_encoder = pickle.load(f)
    model = tf.keras.models.load_model("text_classifier.h5")

    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=max_length, padding='post')
    pred = model.predict(padded_seq)[0][0]

    label = label_encoder.inverse_transform([int(pred > 0.5)])
    print(f"Prediction: {label[0]} (confidence: {pred:.2f})")

# Try prediction
predict_text("I really enjoyed the experience")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 0.6920 - val_accuracy: 0.0000e+00 - val_loss: 0.6972
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step - accuracy: 0.7500 - loss: 0.6892 - val_accuracy: 0.0000e+00 - val_loss: 0.7038
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 1.0000 - loss: 0.6821 - val_accuracy: 0.0000e+00 - val_loss: 0.7088
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.7500 - loss: 0.6778 - val_accuracy: 0.0000e+00 - val_loss: 0.7111
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 1.0000 - loss: 0.6804 - val_accuracy: 0.0000e+00 - val_loss: 0.7136


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
Prediction: 1 (confidence: 0.51)
